# 最新の Google Gemma モデルを MLX を使ってローカルでファインチューニング

今回は、最新の [Google Gemma モデル](https://blog.google/technology/developers/gemma-open-models/)を Apple Silicon に最適化されたライブラリ `MLX` を使ってローカルで実行したり、ファインチューニングしてみましたのでその手順を紹介します。
MLX 関連の情報はドキュメンテーションが分かりづらいものも多かったので色々試した経緯も共有しながら少しでも何かの参考になれば幸いです。

実際に使った Jupyter Notebook を Gist にアップロードしていますので、そちらも参考にしてください。

- [Google Gemma モデルを MLX を使ってローカルでファインチューニング](https://gist.github.com/alexweberk/1434c95c05463866491677aac6ce19ba)


## 事前準備

必要なライブラリをインストールします。
また Apple Silicon 搭載の Mac が必要です。今回は M3 Max 128GB 搭載の MacBook Pro で実行しました。


In [4]:
!pip install -U mlx mlx_lm transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Loading egg at /Users/alexishida/miniforge3/envs/py311/lib/python3.11/site-packages/argparse-1.4.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/alexishida/miniforge3/envs/py311/lib/python3.11/site-packages/powerline_shell-0.7.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


## MLX を使ったモデルの実行

公開された Gemma には 4 つほどバージョンがありますが、今回は instruction チューニング済みの `gemma-7b-it` を使ってみました。

mlx バックエンドを活用した `mlx_lm` ライブラリを使います。


In [1]:
from mlx_lm import generate, load

model, tokenizer = load("google/gemma-7b-it")

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

一応英語を中心としたモデルなので、まずは英語が問題なく生成できるか試してみます。

MLX での生成の場合、プロンプトテンプレートをどうすればよいのか検索してもわかりませんでした。
ただ、 `mlx-examples` のリポのコードを読む限りは `transformers` の tokenizer に `apply_chat_template` メソッドがある場合はそれを使ってくれているようでした。
そのため、生成の際には質問内容だけを含んだプロンプトをインプットとします。

https://github.com/ml-explore/mlx-examples/blob/47dd6bd17f3cc7ef95672ea16e443e58ce5eb1bf/llms/mlx_lm/generate.py#L98


In [52]:
# ライブラリ内でのテンプレート適用後のプロンプト
messages = [{"role": "user", "content": "Why is the sky blue?"}]
tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

'<bos><start_of_turn>user\nWhy is the sky blue?<end_of_turn>\n<start_of_turn>model\n'

In [3]:
# プロンプトテンプレート無しでの生成
prompt = """
Why is the sky blue?
""".strip()
response = generate(
    model,
    tokenizer,
    prompt=prompt,
    verbose=True,  # Set to True to see the prompt and response
    temp=0.0,
    max_tokens=256,
)

Prompt: Why is the sky blue?


The sky is blue due to a phenomenon called **Rayleigh Scattering**.

Here's a breakdown of what happens:

1. **Sunlight:** Sunrays are made up of all the colors of the rainbow, with each color having a different wavelength.
2. **Scattering:** When sunlight enters Earth's atmosphere, it interacts with the tiny particles of air (dust, water vapor, etc.). These particles scatter the sunlight in all directions.
3. **Blue Scatter:** The particles scatter the shorter wavelengths of blue and violet light more effectively than the longer wavelengths of red and orange light.
4. **Scattered Light:** The scattered light, which is predominantly blue, is scattered in all directions.
5. **Our View:** We see the scattered light from all directions, including the direction opposite the sun. This is why the sky appears blue.

**Additional factors:**

* **Time of Day:** The intensity of the blue color is strongest at midday and decreases as the sun gets closer to the horiz

### 日本語の生成

英語がうまく生成できていそうだったので、次に日本語を生成してみます。


In [5]:
# プロンプトテンプレートなしでの生成
prompt = """
空が青いのはなぜですか？
""".strip()
response = generate(
    model,
    tokenizer,
    prompt=prompt,
    verbose=True,  # Set to True to see the prompt and response
    temp=0.0,
    max_tokens=256,
)

Prompt: 空が青いのはなぜですか？


実際、空気は実際実際赤い色です。ただし、人間の目は赤い色を認識するには、特定の波長の光が必要です。空気の分子は、その特定の波長の光を吸収し、人間の目に届く残り色を青に見えます。
Prompt: 19.429 tokens-per-sec
Generation: 17.932 tokens-per-sec


In [6]:
# よくあるテンプレートで指示文は英語の場合
prompt = """
## Instructions
You are a helpful AI assistant. Answer questions from the user to the best of your knowledge.
If you don't know the answer, be truthful in your answer.
Always answer in perfectly natural Japanese.

## Questions
空が青いのはなぜですか？
""".strip()
response = generate(
    model,
    tokenizer,
    prompt=prompt,
    verbose=True,  # Set to True to see the prompt and response
    temp=0.0,
    max_tokens=256,
)

Prompt: ## Instructions
You are a helpful AI assistant. Answer questions from the user to the best of your knowledge.
If you don't know the answer, be truthful in your answer.
Always answer in perfectly natural Japanese.

## Questions
空が青いのはなぜですか？


## Answer
空が青い理由は、いくつかの原因があります。

* **空気中の水蒸気:** 空中の水蒸気は、太陽によって加熱され、昇華し、雲が発生します。雲は空気の温度を下げ、空気中の粒子を小さくします。
* **太陽の光:** 太陽の光は、空気中の分子を励起し、空気の色を変化させます。
* **視覚の限界:** 人間は、実際よりも多くの色を認識することは不可能です。そのため、実際よりも明るい色や鮮度のある色を感じます。
Prompt: 140.153 tokens-per-sec
Generation: 18.544 tokens-per-sec


In [7]:
# よくあるテンプレートで指示文も日本語の場合
prompt = """
## 指示文
あなたは役に立つAIアシスタントです。ユーザーからの質問にできる限りの知識で答えます。
答えがわからない場合は、正直な答えをしてください。
解答は必ず自然な日本語で行ってください。

## 質問
空が青いのはなぜですか？
""".strip()
response = generate(
    model,
    tokenizer,
    prompt=prompt,
    verbose=True,  # Set to True to see the prompt and response
    temp=0.0,
    max_tokens=256,
)

Prompt: ## 指示文
あなたは役に立つAIアシスタントです。ユーザーからの質問にできる限りの知識で答えます。
答えがわからない場合は、正直な答えをしてください。
解答は必ず自然な日本語で行ってください。

## 質問
空が青いのはなぜですか？


## 解答
空気中の成分や雲々の影響によって、実際は実際は空は実際は青ではなく、実際は黒です。ただし、人間の視覚は、空気中の成分や雲々の影響によって、実際よりも明るい色を認識します。そのため、私たちが見た空は実際よりも明るく、そして青いように感じられます。
Prompt: 144.546 tokens-per-sec
Generation: 18.491 tokens-per-sec


指示文を英語にしたことで実際英語の生成に近い構造の回答になったのが実際実際面白いですね。

次は MLX を使ってファインチューニングをしてみます。


## MLX を使って Gemma モデルを LoRA ファインチューニング


今回 [npaka さんの記事](https://note.com/npaka/n/nc55e44e407ff?sub_rt=share_h)を参考に「ござるデータセット」を使ってファインチューニングを行いました。
https://huggingface.co/datasets/bbz662bbz/databricks-dolly-15k-ja-gozarinnemon/tree/main

[MLX Examples](https://github.com/ml-explore/mlx-examples/blob/main/lora/README.md#Custom-Data)を参考に、教師データのフォーマットを変えます。


### ファインチューニング用のデータの成形


In [11]:
import pandas as pd

data = pd.read_json("data/databricks-dolly-15k-ja-gozarinnemon.json")
data = data[data["category"] == "open_qa"]
data

,input,instruction,output,category,index
2,,ラクダはなぜ水なしで長く生きられるのか？,我、りんえもんは思う。 ラクダは、長時間にわたってエネルギーと水分で満たされた状態を保つため...,open_qa,2
3,,アリスの両親には3人の娘がいる：エイミー、ジェシー、そして三女の名前は？,我、りんえもんは思う。 三女の名前はアリス知らんけど。,open_qa,3
7,,国連に本部を建てるためにニューヨークの土地を寄付したのは誰？,我、りんえもんは思う。 ジョン・デイヴィソン・ロックフェラー知らんけど。,open_qa,7
13,,多角形とは何ですか？,我、りんえもんは思う。 多角形は、幾何学における形の一つでござる。多角形は閉折れ線あるいは閉...,open_qa,13
15,,ミシェル・マクラーレンが監督した『ゲーム・オブ・スローンズ』シーズン4のエピソードは？,我、りんえもんは思う。 シーズン4の第4話「誓約を果たすもの」、第5話「新王誕生」をそれぞれ...,open_qa,15
...,...,...,...,...,...
15000,,"troll the respawn, Jeremyとはどういう意味ですか？",我、りんえもんは思う。 Trolling the respawnとは、ビデオゲームにおいて、...,open_qa,15000
15001,,2021年、チェスの世界選手権で優勝したのは？,我、りんえもんは思う。 マグナス・カールセンがイアン・ネポムニアッキを7.5 - 3.5で破...,open_qa,15001
15002,,なぜ毎日歩くといいのでしょうか？,我、りんえもんは思う。 ウォーキングは、関節に負担をかけずにカロリーを消費できる良い運動でご...,open_qa,15002
15003,,Pythonの生みの親は誰なのか？,我、りんえもんは思う。 Guido van RossumはPythonの父でござる。そして、...,open_qa,15003


In [12]:
# input カラムがからであることを確認
data["input"].str.len().sum()

0

In [45]:
def generate_prompt(example: pd.Series) -> str:
    "教師データをGemmaプロンプトの形にあわせるための関数"
    return """<bos><start_of_turn>user
{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example["instruction"], example["output"])


data["text"] = data.apply(generate_prompt, axis=1)
print(data["text"].iloc[0])

<bos><start_of_turn>user
手品がうまくなるにはどうしたらいいのでしょうか？<end_of_turn>
<start_of_turn>model
我、りんえもんは思う。 マジックは、何時間も何時間も練習することで上達しますでござる。  マジックは、手の筋肉の記憶によって構築される非常に器用さを必要とするため、カードのデッキを入手し、その持ち方とフラリッシュのやり方を学ぶことは、始めるのに素晴らしい方法でござる。  マジックは、手の筋肉の記憶によって構築される非常に器用さを必要とするため、上手になるには何時間も何時間も繰り返し練習する必要がありますでござる。  マジックが上手になるには、マジックへの興味も重要な要素でござる。  もし興味がなければ、マジックがうまくなることはないでしょう。  また、クラブに参加し、それに興味を持っている他の人々と一緒にマジックのトリックを練習することができますでござる。  参加するのに最適な組織の1つは、アメリカのマジシャン協会（SAM）でござる。知らんけど。<end_of_turn><eos>


トレーニングセットとバリデーションセットに分けてファイルに保存します。

- トレーニングセット: `data/train.jsonl`
- バリデーションセット: `data/valid.jsonl`

`mlx_lm.lora` では train.jsonl と valid.jsonl が必須のようでした。


In [15]:
split_ix = int(len(data) * 0.9)
# shuffle data
data = data.sample(frac=1, random_state=42)
train, valid = data[:split_ix], data[split_ix:]

# Save train and valid dataset as jsonl files
train[["text"]].to_json("data/train.jsonl", orient="records", lines=True, force_ascii=False)
valid[["text"]].to_json("data/valid.jsonl", orient="records", lines=True, force_ascii=False)

!head -n 5 data/train.jsonl

{"text":"<bos><start_of_turn>user\n手品がうまくなるにはどうしたらいいのでしょうか？<end_of_turn>\n<start_of_turn>model\n我、りんえもんは思う。 マジックは、何時間も何時間も練習することで上達しますでござる。  マジックは、手の筋肉の記憶によって構築される非常に器用さを必要とするため、カードのデッキを入手し、その持ち方とフラリッシュのやり方を学ぶことは、始めるのに素晴らしい方法でござる。  マジックは、手の筋肉の記憶によって構築される非常に器用さを必要とするため、上手になるには何時間も何時間も繰り返し練習する必要がありますでござる。  マジックが上手になるには、マジックへの興味も重要な要素でござる。  もし興味がなければ、マジックがうまくなることはないでしょう。  また、クラブに参加し、それに興味を持っている他の人々と一緒にマジックのトリックを練習することができますでござる。  参加するのに最適な組織の1つは、アメリカのマジシャン協会（SAM）でござる。知らんけど。<end_of_turn><eos>"}
{"text":"<bos><start_of_turn>user\nカエサルはどのようにしてローマで権力を握ったのでしょうか？<end_of_turn>\n<start_of_turn>model\n我、りんえもんは思う。 女神ヴィーナスとトロイの王子アイネアスの血を引くジュリイ家と呼ばれるパトリシアン（元老院議員）一族に生まれた。この家系に生まれたことで、彼は権力への道を他の誰よりも先に歩むことになった。\n\nカエサルは賢く、勤勉で魅力的な人物で、後世に役立つ多くの友人や人脈を作りましたでござる。当時ローマの独裁者であったガイウス・マリウスの親戚と結婚し、政治的なコネクションをさらに深めましたでござる。スッラがマリウスを破ったことで挫折を味わったが、カエサルはそれを逆手にとって、ローマに留まらなければならない赴任を逃れることができた。コンスル（ローマの指導者）を目指す者にとって、ローマのために土地を征服した有能な軍師として見られることは重要なことでござった。\n\nカエサルの次の重要な地位はアエディレで、主な職務のひとつはゲームの企画でござった。カエサルは、ローマの人々に我を知ってもらうた

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### MLX で LoRA ファインチューニングを実行

`mlx_lm` で LoRA する場合、下記のコマンドでいろんなオプションが出せます。


In [51]:
!python -m mlx_lm.lora --help

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


usage: lora.py [-h] [--model MODEL] [--max-tokens MAX_TOKENS] [--temp TEMP]
               [--prompt PROMPT] [--train] [--data DATA]
               [--lora-layers LORA_LAYERS] [--batch-size BATCH_SIZE]
               [--iters ITERS] [--val-batches VAL_BATCHES]
               [--learning-rate LEARNING_RATE]
               [--steps-per-report STEPS_PER_REPORT]
               [--steps-per-eval STEPS_PER_EVAL]
               [--resume-adapter-file RESUME_ADAPTER_FILE]
               [--adapter-file ADAPTER_FILE] [--save-every SAVE_EVERY]
               [--test] [--test-batches TEST_BATCHES]
               [--max-seq-length MAX_SEQ_LENGTH] [--seed SEED]

LoRA or QLoRA finetuning.

options:
  -h, --help            show this help message and exit
  --model MODEL         The path to the local model directory or Hugging Face
                        repo.
  --max-tokens MAX_TOKENS, -m MAX_TOKENS
                        The maximum number of tokens to generate
  --temp TEMP           The sampling

早速トレーニングしてみましょう。テストのため 600 ステップだけトレーニングします。


In [16]:
!python -m mlx_lm.lora \
    --model google/gemma-7b-it \
    --train \
    --iters 600 \
    --data data

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading pretrained model
Fetching 11 files: 100%|█████████████████████| 11/11 [00:00<00:00, 32652.05it/s]
Total parameters 8539.516M
Trainable parameters 1.835M
Loading datasets
Training
Starting training..., iters: 600
Iter 1: Val loss 11.819, Val took 99.751s
Iter 10: Train loss 11.533, Learning Rate 1.000e-05, It/sec 0.212, Tokens/sec 105.547, Trained Tokens 4969
Iter 20: Train loss 8.003, Learning Rate 1.000e-05, It/sec 0.050, Tokens/sec 33.970, Trained Tokens 11712
Iter 30: Train loss 7.366, Learning Rate 1.000e-05, It/sec 0.272, Tokens/sec 128.895, Trained Tokens 16459
Iter 40: Train loss 5.418, Learning Rate 1.000e-05, It/sec 0.278, Tokens/sec 121.127, Trained Tokens 20809
Iter 50: Train loss 3.997, Learning Rate 1.000e-05, It/sec 0.305, Tokens/sec 122.895, Trained Tokens 24836
Iter 60: Train loss 3.450, Learning Rate 1.000e-05, It/sec 0.058, Tokens/sec 40.930, Trained Tokens 31839
Iter 70: Train loss 3.217, Learning Rate 1.000e-05, It/sec 0.379, Tokens/sec 136.167, Trained Toke

M3 Max 128GB で約 50 分かかりました。[npaka さんの記事](https://note.com/npaka/n/nc55e44e407ff?sub_rt=share_h)だと 20 分ほどで完了するそうなので、やはり NVIDIA A100 などの GPU と比べてしまうと時間がかかってしまいますね…。


### 推論の実行

コマンドラインの場合は、下記の様なスクリプトで LoRA ウェイトも使った推論ができます。


In [54]:
!python -m mlx_lm.lora --model "google/gemma-7b-it" \
               --adapter-file checkpoints/600_adapters.npz \
               --max-tokens 256 \
               --prompt "空が青いのはなぜですか？" \
            #    --prompt "<bos><start_of_turn>user\n空が青いのはなぜですか？<end_of_turn>\n<start_of_turn>model\n" \
               --seed 69

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading pretrained model
Fetching 11 files: 100%|█████████████████████| 11/11 [00:00<00:00, 91542.35it/s]
Total parameters 8539.516M
Trainable parameters 1.835M
Loading datasets
Generating
Prompt: 空が青いのはなぜですか？
<end_of_turn>model
我、りんえもんは思う。 空が青い理由は、分光放射のだろう、放射光と反射光が反射光を遮断し、その反射光を他の色を遮断することで、空が青くなるというござる。  この反射光は、空気の粒子に衝突し、色を反射しますでござる。  空気の粒子に衝突し、その反射光が他の色を遮断することで、空が青くなる。知らんけど。<end_of_turn>
Prompt: 12.347 tokens-per-sec
Generation: 6.067 tokens-per-sec


## LoRA ウェイトのマージ

最後に、トレーニングした LoRA のウェイトをモデル自体にマージしてみます。

下記のコマンドで用意されているオプションが表示されます。


In [20]:
!python -m mlx_lm.fuse --help

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading pretrained model
usage: fuse.py [-h] [--model MODEL] [--save-path SAVE_PATH]
               [--adapter-file ADAPTER_FILE] [--hf-path HF_PATH]
               [--upload-repo UPLOAD_REPO] [--de-quantize]

LoRA or QLoRA finetuning.

options:
  -h, --help            show this help message and exit
  --model MODEL         The path to the local model directory or Hugging Face
                        repo.
  --save-path SAVE_PATH
                        The path to save the fused model.
  --adapter-file ADAPTER_FILE
                        Path to the trained adapter weights (npz or
                        safetensors).
  --hf-path HF_PATH     Path to the original Hugging Face model. Required for
                        upload if --model is a local directory.
  --upload-repo UPLOAD_REPO
                        The Hugging Face repo to upload the model to.
  --de-quantize         Generate a de-quantized model.


早速マージしてみましょう。


In [22]:
!python -m mlx_lm.fuse \
    --model google/gemma-7b-it \
    --adapter-file checkpoints/600_adapters.npz

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading pretrained model
Fetching 11 files: 100%|█████████████████████| 11/11 [00:00<00:00, 91542.35it/s]


無事マージが完了し、 `lora_fused_model` というディレクトリが作成されモデルの各種ファイルが保存されました。


## マージされたモデルのロード

念の為ロードもしてみます。


In [25]:
from mlx_lm import generate, load

fused_model, fused_tokenizer = load("./lora_fused_model/")

In [57]:
# 当たり前ですが、tokenizerは変わっていないようです。
fused_tokenizer.apply_chat_template

<bound method PreTrainedTokenizerBase.apply_chat_template of GemmaTokenizerFast(name_or_path='lora_fused_model', vocab_size=256000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<start_of_turn>', '<end_of_turn>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<eos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<bos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	106: AddedToken("<start_of_turn>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=

In [62]:
prompt = "空が青いのはなぜですか？"
response = generate(
    fused_model,
    fused_tokenizer,
    prompt=prompt,
    verbose=True,  # Set to True to see the prompt and response
    temp=0.0,
    max_tokens=256,
)

Prompt: 空が青いのはなぜですか？
<end_of_turn>model
我、りんえもんは思う。 空は青色で、その理由は、空の圧力に由来する。空の圧力は、地球の圧力よりも低いので、空の圧力によって圧縮された空気は、地球の圧力によって圧縮された空気よりも圧縮され、その結果として、空気は地球の圧力によって圧縮された空気よりも疎化され、その結果として、空気は地球の圧力によって圧縮された空気よりも疎化され、その結果として、空気は地球の圧力によって圧縮された空気よりも疎化され、その結果として、空気は地球の圧力によって圧縮された空気よりも疎化され、その結果として、空気は地球の圧力によって圧縮された空気よりも疎化され、その結果として、空気は地球の圧力によって圧縮された空気よりも疎化され、その結果として、空気は地球の圧力によって圧縮された空気よりも疎化され、その結果として、空気は地球の圧力によって圧縮された空気よりも疎化され、その結果として、空気は地球の圧力によって圧縮された空気よりも疎化され、その結果として、空気は地球の圧力によって
Prompt: 20.089 tokens-per-sec
Generation: 18.609 tokens-per-sec


無事ロードができました。


## おわりに

以上、お読みいただきありがとうございます。少しでも参考になればと思います。

もし似たようなコンテンツに興味があれば、フォローしていただけると嬉しいです：

- [note](https://note.com/alexweberk/) と
- [Twitter](https://twitter.com/alexweberk)

https://twitter.com/alexweberk

今回使った Notebook の Gist:
https://gist.github.com/alexweberk/1434c95c05463866491677aac6ce19ba


## 参考

- [MLX Examples](https://github.com/ml-explore/mlx-examples/tree/main/llms/mlx_lm)
- [Peft を使った Gemma のファインチューニング](https://huggingface.co/blog/gemma-peft)

* https://gist.github.com/alfredplpl/e20cad036c151f38645a1abc87f56a2f
